In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statistics as es 
import csv
import re 
import warnings
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## ENEM  DATASET

In [37]:
#Choose just useful features for us 
fenem= ['NU_INSCRICAO','CO_ESCOLA','Q001','Q002', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
enem17a=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2017.csv", sep=';',encoding="iso-8859-2", usecols = fenem)
enem18a=pd.read_csv("~/data/Enem/MICRODADOS_ENEM_2018.csv", sep=';',encoding="iso-8859-2", usecols = fenem)

In [38]:
category=(['NU_INSCRICAO','CO_ESCOLA','Q001','Q002'])
#enem17a[category]= enem17a[category].astype(str)

In [39]:
enem17 = enem17a.copy()
enem18 = enem18a.copy()


In [40]:
#How many Students left? How many schools are they in?
print ("There are", enem17.shape[0], "students",  "of", enem17.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2017")
#enem17.head()

print ("There are", enem18.shape[0], "students",  "of", enem18.CO_ESCOLA.value_counts().count() ,"Schools in ENEM 2018")
#enem18.head()

There are 6731341 students of 33834 Schools in ENEM 2017
There are 5513747 students of 32253 Schools in ENEM 2018


#### Set last quartile of average scores as the best students

In [41]:
#first, drop out those whithout schools 
enem17.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])
enem18.dropna(inplace = True, axis=0, subset=['CO_ESCOLA'])

In [42]:
#Second, drop out insufficient scores 
fields = (['NU_NOTA_CN', 'NU_NOTA_CN','NU_NOTA_CN','NU_NOTA_CN', 'NU_NOTA_REDACAO'])
# With NaN
enem17.dropna(inplace = True, axis=0, subset=fields)
enem18.dropna(inplace = True, axis=0, subset=fields)
# With 0
enem17 = enem17[~(enem17[fields] == 0).any(axis=1)]
enem18 = enem18[~(enem18[fields] == 0).any(axis=1)]

In [43]:
#Creating average of students in all fields
enem17['AVG_SCORE'] = (enem17.NU_NOTA_CN + enem17.NU_NOTA_CH + enem17.NU_NOTA_LC + enem17.NU_NOTA_MT + enem17.NU_NOTA_REDACAO)/5
enem18['AVG_SCORE'] = (enem18.NU_NOTA_CN + enem18.NU_NOTA_CH + enem18.NU_NOTA_LC + enem18.NU_NOTA_MT + enem18.NU_NOTA_REDACAO)/5

In [44]:
#Creating the average of all schools by each studentes score
enem17['AVG_SCORE_SCHOOL'] = enem17.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 
enem18['AVG_SCORE_SCHOOL'] = enem18.groupby(['CO_ESCOLA'])['AVG_SCORE'].transform('mean') 

In [45]:
#Sorting out the shcool's scores in quartiles. The upper quartile will be the Good Schools.
enem17['TARGET'] = pd.qcut (enem17.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 
enem18['TARGET'] = pd.qcut (enem18.AVG_SCORE_SCHOOL, 4, labels = [1,2,3,4]).map(lambda x : 0 if x!=4 else 1) 


In [46]:
#It's right? 
print((enem17.TARGET==0).sum()/(enem17.TARGET.count())*100, '% lowers quartis in ENEM 2017')
print((enem18.TARGET==0).sum()/(enem18.TARGET.count())*100, '% lowers quartis in ENEM 2018')

75.00009542589339 % lowers quartis in ENEM 2017
75.00310581817226 % lowers quartis in ENEM 2018


In [213]:
enem17.head(2)

,NU_INSCRICAO,CO_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q001,Q002,AVG_SCORE,AVG_SCORE_SCHOOL,TARGET
47,170001663652,13079220.0,540.7,583.0,472.9,405.1,580.0,E,G,516.34,593.840800,1
84,170001664222,29422418.0,396.4,401.6,350.7,494.2,280.0,G,G,384.58,467.525714,0


In [214]:
enem18.head(2)

,NU_INSCRICAO,CO_ESCOLA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,NU_NOTA_REDACAO,Q001,Q002,AVG_SCORE,AVG_SCORE_SCHOOL,TARGET
8,180008257068,23077140.0,411.8,429.8,445.2,459.0,340.0,D,D,417.16,483.509028,0
16,180007766857,23265795.0,478.5,513.3,564.7,514.9,580.0,C,C,530.28,479.086667,0


## TEACHERS CENSUS DATASET 

In [77]:
#Choose just useful features for us now, improve memory use
fcenso = ['CO_PESSOA_FISICA','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

#Reading data of Teachers in Censo 2017
CO17=pd.read_csv("~/data/censo2017/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NE17=pd.read_csv("~/data/censo2017/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
NT17=pd.read_csv("~/data/censo2017/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SE17=pd.read_csv("~/data/censo2017/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)
SU17=pd.read_csv("~/data/censo2017/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso)

#There are differences in feature names
fcenso18 = ['ID_DOCENTE','NU_ANO_CENSO','IN_ESPECIALIZACAO','IN_MESTRADO','IN_DOUTORADO','CO_ENTIDADE']

CO18=pd.read_csv("~/data/censo2018/DOCENTES_CO.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NE18=pd.read_csv("~/data/censo2018/DOCENTES_NORDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
NT18=pd.read_csv("~/data/censo2018/DOCENTES_NORTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SE18=pd.read_csv("~/data/censo2018/DOCENTES_SUDESTE.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)
SU18=pd.read_csv("~/data/censo2018/DOCENTES_SUL.CSV", sep='|', encoding="iso-8859-2", usecols = fcenso18)


In [78]:
#Concatening the subsets and creating a unique dataset, than transform all features and categories, as it are.
censo17a= pd.concat([CO17, NE17, NT17, SE17, SU17])
censo17 = censo17a.copy()

censo18a= pd.concat([CO18, NE18, NT18, SE18, SU18])
censo18 = censo18a.copy()
#censo17=censo17.astype('category')

Get only teachers who work in secondary schools (based on ENEM)

In [79]:
#Some informations
print ("There are", censo17.drop_duplicates('CO_PESSOA_FISICA').shape[0], "Teachers",  "at", censo17.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2017")

print ("There are", censo18.drop_duplicates('ID_DOCENTE').shape[0], "Teachers",  "at", censo18.CO_ENTIDADE.value_counts().count() ,"Schools in Census 2018")

There are 2603186 Teachers at 185925 Schools in Census 2017
There are 2581297 Teachers at 183746 Schools in Census 2018


In [80]:
#It's necessary drop duplicates whose are teachers in the same school but in different classes. 
#This way, the teacher appear just one time in each school. 
censo17.drop_duplicates(subset=['CO_PESSOA_FISICA', 'CO_ENTIDADE'], inplace=True)
censo18.drop_duplicates(subset=['ID_DOCENTE', 'CO_ENTIDADE'], inplace=True)

In [83]:
#getting the schools represented in ENEM
schoolsEnem17 = enem17.drop_duplicates('CO_ESCOLA')
schoolsEnem17 = schoolsEnem17[(['CO_ESCOLA','TARGET'])]
#schoolsEnem17 = schoolsEnem17.astype(str)

schoolsEnem18 = enem18.drop_duplicates('CO_ESCOLA')
schoolsEnem18 = schoolsEnem18[(['CO_ESCOLA','TARGET'])]
#schoolsEnem18 = schoolsEnem18.astype(str)


In [84]:
#schoolsEnem18
#censo18

In [85]:
#Merge the two datasets, whereas that only teachers who work in school represented in ENEM 17 and 18
censo17= pd.merge(schoolsEnem17, censo17, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

censo18= pd.merge(schoolsEnem18, censo18, left_on='CO_ESCOLA', right_on='CO_ENTIDADE', how='inner')

In [86]:
print(censo17.isnull().sum())
print(censo18.isnull().sum())

CO_ESCOLA                 0
TARGET                    0
NU_ANO_CENSO              0
CO_PESSOA_FISICA          0
IN_ESPECIALIZACAO    117764
IN_MESTRADO          117764
IN_DOUTORADO         117764
CO_ENTIDADE               0
dtype: int64
CO_ESCOLA                0
TARGET                   0
NU_ANO_CENSO             0
ID_DOCENTE               0
IN_ESPECIALIZACAO    97672
IN_MESTRADO          97672
IN_DOUTORADO         97672
CO_ENTIDADE              0
dtype: int64


In [87]:
#Missing values in these cases are "0" 
censo17['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo17['IN_MESTRADO'].fillna(0.0, inplace=True)
censo17['IN_DOUTORADO'].fillna(0.0, inplace=True)

censo18['IN_ESPECIALIZACAO'].fillna(0.0, inplace=True)
censo18['IN_MESTRADO'].fillna(0.0, inplace=True)
censo18['IN_DOUTORADO'].fillna(0.0, inplace=True)

In [88]:
#In database have different patterns to represent level of education. ex: Master has signed in two different ways, or marked in Master and Especialization or just in Master.
def education_level(df):
    if (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(0)
        df['TITULACAO']= "0"
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==0.0) & (df.IN_DOUTORADO ==0.0):
        #print(1)
        df['TITULACAO']= "1"
    elif (df.IN_ESPECIALIZACAO==1.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= "2"
    elif (df.IN_ESPECIALIZACAO==0.0) & (df.IN_MESTRADO==1.0) & (df.IN_DOUTORADO ==0.0):
        #print(2)
        df['TITULACAO']= "2"
    else:
        #print(3)
        df['TITULACAO']= "3"  
    return df

In [89]:
censo17=censo17.apply(education_level, axis=1)

censo18=censo18.apply(education_level, axis=1)

In [97]:
censo17['TARGET'] = censo17['TARGET'].astype('int64')

In [98]:
censo17.head(2)

,CO_ESCOLA,TARGET,NU_ANO_CENSO,CO_PESSOA_FISICA,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,CO_ENTIDADE,TITULACAO
0,13079220.0,1,2017.0,1.222391e+11,1.0,0.0,0.0,13079220.0,1
1,13079220.0,1,2017.0,1.150041e+11,0.0,0.0,0.0,13079220.0,0


In [91]:
censo18.head(2)

,CO_ESCOLA,TARGET,NU_ANO_CENSO,ID_DOCENTE,IN_ESPECIALIZACAO,IN_MESTRADO,IN_DOUTORADO,CO_ENTIDADE,TITULACAO
0,23077140.0,0,2018,00E3AA8026744AEEC6D35E11B082B9D9,0.0,0.0,0.0,23077140,0
1,23077140.0,0,2018,0FA5A09CB34A5FEC043158EE31F812F6,0.0,0.0,0.0,23077140,0


##### We will transform the Categorical ENEM database features from student grain to school grain (Decision Grain).  Instead the pratical use of MODE, it will use the regression on the attribute distribution, having its histogram relative frequencies as input and the  decision level TARGET as output.  

In [199]:
#Creating the dataframe with the relative frequencies and target. Ready to applying regeression
def relative_frequencies(df, colum):
    root = (['CO_ESCOLA', 'TARGET'])
    df = df [root + [colum]]
    df = pd.get_dummies(df, columns=[colum])
    filter_col = [col for col in df if col.startswith(colum)]
    print (filter_col)
    for x in filter_col:            
        df[x] = df.groupby(df['CO_ESCOLA'])[x].transform(sum)/ df.groupby(df['CO_ESCOLA'])[x].transform('count')
        
    return df

In [369]:
def LR_student_to_school (df1, df2, columns):
    result = pd.DataFrame()
    for x in columns:
        temp = relative_frequencies(df1,df1[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True)
        temp = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp.loc[:, temp.columns != 'TARGET']
        temp_TARGET = temp.loc[:, temp.columns == 'TARGET']

        
        LR_temp = LogisticRegression()
        LR_temp.fit(temp_DATA, temp_TARGET)
      
        
        temp = relative_frequencies(df2,df2[x].name)
        temp.drop_duplicates('CO_ESCOLA', inplace=True) 
        temp = temp.reset_index(drop=True)
        temp2 = temp.loc[:, ~temp.columns.isin(['CO_ESCOLA'])]
        temp_DATA = temp2.loc[:, temp2.columns != 'TARGET']
        
        result[x] = LR_temp.predict_proba(temp_DATA)[:,1]
        
       
    result = pd.merge(result, temp.CO_ESCOLA, left_index=True, right_index=True)
    
    return result, LR_temp

In [375]:
category_enem = (['Q001', 'Q002'])
trans_enem_att, model_enem =LR_student_to_school(enem17, enem18, category_enem)

['Q001_A', 'Q001_B', 'Q001_C', 'Q001_D', 'Q001_E', 'Q001_F', 'Q001_G', 'Q001_H']
['Q001_A', 'Q001_B', 'Q001_C', 'Q001_D', 'Q001_E', 'Q001_F', 'Q001_G', 'Q001_H']
['Q002_A', 'Q002_B', 'Q002_C', 'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H']
['Q002_A', 'Q002_B', 'Q002_C', 'Q002_D', 'Q002_E', 'Q002_F', 'Q002_G', 'Q002_H']


In [389]:
trans_enem_att.head()

,Q001,Q002,CO_ESCOLA
0,0.101042,0.026863,23077140.0
1,0.011041,0.008507,23265795.0
2,0.013038,0.007904,25077759.0
3,0.045325,0.042808,22022040.0
4,0.166826,0.095757,35907224.0


In [424]:
dat = dat.loc[:, dat.columns != 'TARGET']


In [409]:
#dat2 = relative_frequencies(dat, dat.Q002.name)
dat = dat.loc[enem18.CO_ESCOLA ==23077140.0]
dat = dat.loc[:, ~dat.columns.isin(['CO_ESCOLA'])]
dat2 = dat2.loc[:, dat2.columns != 'TARGET']
dat2

['Q002_B', 'Q002_C', 'Q002_D', 'Q002_E']


,Q002_B,Q002_C,Q002_D,Q002_E
8,0.006944,0.277778,0.708333,0.006944


In [425]:
dat

,Q002_A,Q002_B,Q002_C,Q002_D,Q002_E,Q002_F,Q002_G,Q002_H
8,0.0,0.006944,0.277778,0.708333,0.006944,0.0,0.0,0.0


In [429]:
model_enem.predict_proba(dat)

array([[0.97313662, 0.02686338]])

In [373]:
category_censo = (['TITULACAO'])
trans_censo_att, model_censo=LR_student_to_school(censo17, censo18, category_censo)

['TITULACAO_0', 'TITULACAO_1', 'TITULACAO_2', 'TITULACAO_3']
['TITULACAO_0', 'TITULACAO_1', 'TITULACAO_2', 'TITULACAO_3']


In [387]:
trans_censo_att.head()

,TITULACAO,CO_ESCOLA
0,0.262913,23077140.0
1,0.225096,23265795.0
2,0.348830,25077759.0
3,0.239566,22022040.0
4,0.250300,35907224.0


In [383]:
model_censo.coef_

array([[-1.71874298, -4.0725389 ,  6.13096251,  0.65454639]])

In [384]:
model_censo.intercept_

array([0.99422702])